<a href="https://colab.research.google.com/github/Murakami-Shoya/2021_school/blob/master/2021_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import glob, random
from sklearn.preprocessing import OneHotEncoder

In [3]:
# パスは変更のこと
path = "/content/drive/MyDrive/leap_dataset/hand_data_2021_test/"

## データをすべてまとめる関数を作る

In [4]:
def set_data(gesture_name):

    random.seed(123)

    # 1ジェスチャーが何フレームかでデータをスライスする量が変わる
    frame_num = 50

    output_df = pd.DataFrame()

    all_files = glob.glob(path + gesture_name + "*.csv")
    # ランダムに並べ替える
    shuffled_all_files = random.sample(all_files, len(all_files))

    for one_file in all_files:
        one_df = pd.read_csv(one_file, sep=',')[:frame_num]
        output_df = pd.concat([output_df, one_df])

    # output_df.to_csv("./all_data_folder/all_cut_data.csv",header=False,  index=False)
    return output_df

### 手のひら座標に変換する関数


引数はndarray
返り値もndarray

In [5]:
def normalize(X):
            
    hands = X.reshape([-1, 27, 3])
    joints = []

    for hand in hands:
        palm_position = hand[0]
        direction = hand[1]
        palm_y = hand[2]
    
        palm_x = np.cross(direction, palm_y,)

        R = np.stack([palm_x, palm_y, direction])
        R = np.linalg.inv(R)

        joint = hand[3:]
        joint = joint - palm_position

        joint = np.dot(joint, R)

        palm_position = np.dot(np.array([0.0, 0.0, 0.0]), R)
        direction = np.dot(direction, R)
        palm_y = np.dot(palm_y, R)

        joint = np.vstack([palm_position, direction, palm_y, joint])

        joints.append(joint)
    # 返り値を1フレームごとのnumpy配列に変更
    joints = np.array(joints).reshape([-1, 27*3])

    return joints

## load_hand_data

引数はファイル名のリスト、そのインデックス番号、学習用データ数

返り値はndarrayが4つのタプル

In [6]:
def load_hand_data(gesture_df_list, index, train_rate=0.8):
    output_size = len(gesture_df_list)

    train_X = []
    train_y = []
    test_X = []
    test_y = []

    for i in range(output_size):
        # data = gesture_df_list[i].values.reshape([-1, 27, 3])
        data = gesture_df_list[i].values
        data_len = len(data)
        teacher = np.array([[1 if k == index[i] else 0 for k in range(output_size)] for _ in range(data_len)]) #教師データの生成(ワンホットベクトル)
        
        # 学習用と検証用に分ける(train_test_split)
        train_frame_num = int(data_len * train_rate)
        train_data = data[:train_frame_num]
        test_data = data[train_frame_num:]
        train_teacher = teacher[:train_frame_num]
        test_teacher = teacher[train_frame_num:]
        # 全てのジェスチャーを結合
        train_X += train_data.tolist()
        test_X += test_data.tolist()
        train_y += train_teacher.tolist()
        test_y += test_teacher.tolist()
    # リストからndarrayに変換
    train_X, test_X, train_y, test_y = np.array(train_X), np.array(test_X), np.array(train_y), np.array(test_y)

    # 入力データは全て正規化を行う
    train_X = normalize(train_X)
    test_X = normalize(test_X)
    # 指に関節座標のみ抽出
    train_X = train_X[:, 3*3:]
    test_X = test_X[:, 3*3:]

    return train_X, test_X, train_y, test_y